# Comparative Stock & Revenue Dashboard

This notebook extracts historical stock data and revenue data for Tesla and GameStop using yfinance and web scraping, and then visualizes the data using interactive Plotly dashboards.

## Table of Contents

1. **Question 1:** Extract Tesla Stock Data using yfinance
2. **Question 2:** Extract Tesla Revenue Data via Web Scraping
3. **Question 3:** Extract GameStop Stock Data using yfinance
4. **Question 4:** Extract GameStop Revenue Data via Web Scraping
5. **Question 5:** Plot Tesla Stock & Revenue Dashboard
6. **Question 6:** Plot GameStop Stock & Revenue Dashboard

In [ ]:
# Imports and Setup
!pip install yfinance bs4 html5lib lxml pandas matplotlib plotly

import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
%matplotlib inline
from plotly.subplots import make_subplots
import plotly.graph_objects as go

## Question 1: Extract Tesla Stock Data using yfinance

This cell creates a Tesla ticker object, extracts historical stock data with a maximum period, resets the index, and displays the first five rows.

In [ ]:
# Question 1

tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)
print("Question 1: Tesla Stock Data - First 5 rows:")
tesla_data.head()

## Question 2: Extract Tesla Revenue Data via Web Scraping

This cell downloads the Tesla revenue webpage, extracts the revenue table using pandas.read_html, cleans the data, and displays the last five rows.

In [ ]:
# Question 2
from io import StringIO

url_tesla_revenue = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data_tesla = requests.get(url_tesla_revenue).text

# Wrap the HTML in a StringIO object to avoid the FutureWarning
tesla_revenue_list = pd.read_html(StringIO(html_data_tesla))

# Assume the first table is the revenue table
tesla_revenue = tesla_revenue_list[0]

# Print columns for debugging
print("Columns in Tesla revenue table:", tesla_revenue.columns.tolist())

# Rename columns if necessary
if list(tesla_revenue.columns) != ["Date", "Revenue"]:
    tesla_revenue.columns = ["Date", "Revenue"]

# Clean the Revenue column: remove "$" and commas, then convert to numeric
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace(r",|\$", "", regex=True)
tesla_revenue["Revenue"] = pd.to_numeric(tesla_revenue["Revenue"], errors="coerce")

print("Question 2: Tesla Revenue Data - Last 5 rows:")
tesla_revenue.tail()

## Question 3: Extract GameStop Stock Data using yfinance

This cell creates a GameStop ticker object, extracts historical stock data with a maximum period, resets the index, and displays the first five rows.

In [ ]:
# Question 3

gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)
print("Question 3: GameStop Stock Data - First 5 rows:")
gme_data.head()

## Question 4: Extract GameStop Revenue Data via Web Scraping

This cell downloads the GameStop revenue webpage, extracts the revenue table, cleans the data, and displays the last five rows.

In [ ]:
# Question 4
url_gme_revenue = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data_gme = requests.get(url_gme_revenue).text

# Wrap the HTML in a StringIO object to avoid the FutureWarning
from io import StringIO

gme_revenue_list = pd.read_html(StringIO(html_data_gme))

# Print columns for debugging
print("Columns in GameStop revenue table:", gme_revenue_list[0].columns.tolist())

# Assume the first table is the revenue table
gme_revenue = gme_revenue_list[0]
if "Revenue" not in gme_revenue.columns:
    gme_revenue.columns = ["Date", "Revenue"]

# Clean the Revenue column
gme_revenue["Revenue"] = gme_revenue["Revenue"].str.replace(r",|\$", "", regex=True)
gme_revenue["Revenue"] = pd.to_numeric(gme_revenue["Revenue"], errors="coerce")

print("Question 4: GameStop Revenue Data - Last 5 rows:")
gme_revenue.tail()

## Define the make_graph Function

This function creates a dashboard with two subplots using Plotly: one for historical stock closing price and one for historical revenue data. It filters the data up to specific dates and then plots the graphs.

In [ ]:
def make_graph(stock_data, revenue_data, stock):
    # Convert the Date columns to datetime
    stock_data["Date"] = pd.to_datetime(stock_data["Date"], errors="coerce")
    revenue_data["Date"] = pd.to_datetime(revenue_data["Date"], errors="coerce")
    
    # Filter data up to specified dates
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    
    # Create subplots
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        subplot_titles=("Historical Share Price", "Historical Revenue"),
                        vertical_spacing=0.3)
    
    # Add stock price trace
    fig.add_trace(go.Scatter(
        x=stock_data_specific.Date,
        y=stock_data_specific.Close.astype("float"),
        name="Share Price"
    ), row=1, col=1)
    
    # Add revenue trace
    fig.add_trace(go.Scatter(
        x=revenue_data_specific.Date,
        y=revenue_data_specific.Revenue.astype("float"),
        name="Revenue"
    ), row=2, col=1)
    
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price (USD)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue (USD Millions)", row=2, col=1)
    
    fig.update_layout(
        height=900,
        title=stock,
        showlegend=False,
        xaxis_rangeslider_visible=True
    )
    
    fig.show()


## Question 5: Plot Tesla Stock & Revenue Dashboard

This cell calls the `make_graph` function to display the Tesla dashboard graph with the historical stock price and revenue data.

In [ ]:
# Question 5
make_graph(tesla_data, tesla_revenue, "Tesla")

## Question 6: Plot GameStop Stock & Revenue Dashboard

This cell calls the `make_graph` function to display the GameStop dashboard graph with the historical stock price and revenue data.

In [ ]:
# Question 6
make_graph(gme_data, gme_revenue, "GameStop")